In [1]:
import cv2
import numpy as np

# Initialize global variables
drawing = False  # True if mouse is down, used to track drawing state
points = []  # List to store points of the polygon

# Mouse callback function to draw polygon and trigger mask creation
def draw_polygon(event, x, y, flags, param):
    global points, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        points.append((x, y))  # Add point on left mouse button down

    elif event == cv2.EVENT_LBUTTONDBLCLK:
        # Finalize drawing on double click
        drawing = False
        if len(points) > 2:
            mask = create_mask(frame.shape, points)  # Create mask based on polygon points
            cv2.fillPoly(frame, [np.array(points, np.int32)], (0, 0, 0))  # Apply mask to the frame visually
            save_mask(mask, 'mask.png')  # Save the created mask
        points.clear()  # Clear the points after drawing the polygon

# Function to create binary mask from polygon points
def create_mask(shape, polygon_points):
    """
    Creates a binary mask where the polygon area is 1 and the outside is 0.
    :param shape: Tuple of the dimensions of the frame (height, width, channels).
    :param polygon_points: List of points (x, y) that make up the polygon.
    :return: Binary mask as a numpy array.
    """
    mask = np.zeros(shape[:2], dtype=np.uint8)  # Initialize mask with zeros
    cv2.fillPoly(mask, [np.array(polygon_points, np.int32)], 1)  # Fill polygon area in the mask with 1
    return mask

# Function to save the binary mask to a file
def save_mask(mask, filename):
    """
    Saves the binary mask to a file.
    :param mask: Binary mask as a numpy array.
    :param filename: Filename for the saved mask image.
    """
    cv2.imwrite(filename, mask * 255)  # Convert binary mask (0,1) to image format (0,255) and save
    print(f"Mask saved as {filename}")

# Function to capture video, display it, and allow drawing of polygons
def mask_video(video_path):
    global frame  # To modify the frame within the callback
    cap = cv2.VideoCapture(video_path)  # Capture video from the specified path
    cv2.namedWindow('Video')
    cv2.setMouseCallback('Video', draw_polygon)  # Set mouse callback for drawing

    while True:
        ret, frame = cap.read()  # Read frames from the video
        if not ret:
            print("Failed to grab frame")
            break

        if len(points) > 1:
            # Draw the current polygon being drawn
            tmp_frame = frame.copy()
            cv2.polylines(tmp_frame, [np.array(points, np.int32)], False, (0, 255, 0), 1)
            cv2.imshow('Video', tmp_frame)
        else:
            cv2.imshow('Video', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to quit the application
            break

    cap.release()
    cv2.destroyAllWindows()
    
    
# Example usage
video_path = 'https://chiefcam.com/video/hls/live/1080p/index.m3u8'
#'https://chiefcam.com/video/hls/live/1080p/index.m3u8'

mask_video(video_path)

Mask saved as mask.png
Mask saved as mask.png


In [ ]:
import cv2
import numpy as np

# Load the image in grayscale
image = cv2.imread('mask.png', cv2.IMREAD_GRAYSCALE)

if image is None:
    print('Failed to load image')
else:
    # Threshold the image to make sure it's binary
    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Assuming you want to work with the largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Create an empty mask for the distance transform
        mask = np.zeros_like(image)
        cv2.drawContours(mask, [largest_contour], -1, 255, -1)  # Draw filled contour
        
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        print(f"x,y,w,h: {x,y, w, h}")
        
        # Apply distance transform
        dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(dist_transform)
        print( min_val, max_val, min_loc, (max_loc))
        # The maximum location is the center of the circle
        center = (max_loc)
        radius = int(max_val)  # The maximum value is the radius of the circle
        
        # Define the cropping rectangle
        radius = int(max_val)
        top_left = (max(max_loc[0] - radius, 0), max(max_loc[1] - radius, 0))
        bottom_right = (min(max_loc[0] + radius, mask.shape[1] - 1), min(max_loc[1] + radius, mask.shape[0] - 1))
        
        print(f'top_left:{top_left}, bottom_right: {bottom_right}')

        # Draw the circle on the original image
        output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert to BGR to draw colored shapes
        cv2.circle(output_image, center, radius, (0, 255, 0), 2)  # Draw the circle in green with thickness 2

        # Optionally, show or save the result
        cv2.imshow('Inscribed Circle', output_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print('No contours found')


In [ ]:
import os , subprocess

def set_video_bitrate(input_video_path, output_video_path, bitrate_kbps):
    """
    Set the bitrate of a video using FFmpeg. If the output file already exists, a new file with a suffix is created instead.

    Parameters:
    - input_video_path: Path to the input video file.
    - output_video_path: Path where the output video with the new bitrate will be saved.
    - bitrate_kbps: Desired video bitrate in kbps.
    """
    try:
        # Check if the output file already exists
        original_output_path = output_video_path
        counter = 1
        while os.path.exists(output_video_path):
            # Split the output path into directory, base name, and extension
            dir_name, base_name = os.path.split(original_output_path)
            file_name, file_extension = os.path.splitext(base_name)
            
            # Append a suffix to make the file name unique
            output_video_path = os.path.join(dir_name, f"{file_name}_{counter}{file_extension}")
            counter += 1

        # Build the FFmpeg command
        command = [
            'ffmpeg',
            '-i', input_video_path,
            '-b:v', f'{bitrate_kbps}k',
            '-bufsize', f'{int(bitrate_kbps * 2)}k',  # Optional: Adjust buffer size
            output_video_path
        ]

        # Execute the FFmpeg command
        subprocess.run(command, check=True)

        print(f"Video with bitrate {bitrate_kbps} kbps saved as {output_video_path}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to set bitrate: {e}")
        
        
set_video_bitrate('./plots/GIFS/frame_2024-03-20_0_image.mp4', './plots/GIFS/converted/frame_2024-03-20_0_image.mp4', 2000)

In [ ]:
import cv2
import numpy as np

# Initialize the ORB detector
orb = cv2.ORB_create()

# Initialize the MOG2 background subtractor
backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

# Open the video file or capture device
cap = cv2.VideoCapture('https://chiefcam.com/video/hls/live/1080p/index.m3u8')

ret, frame1 = cap.read()
if not ret:
    print("Failed to read the video stream")
    cap.release()
    cv2.destroyAllWindows()
    exit()

prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Initialize an empty frame for the running weighted average
avg_frame = None
alpha = 0.1  # Weight for the running average

# Initialize variables for ORB feature tracking
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# Load the uploaded mask and convert it to grayscale
uploaded_mask = cv2.imread('mask.png', cv2.IMREAD_GRAYSCALE)
if uploaded_mask is None:
    print("Failed to load mask.png")
    cap.release()
    cv2.destroyAllWindows()
    exit()
shifts_x = []
shifts_y = []
import matplotlib.pyplot as plt
# Initialize plotting
plt.ion()  # Turn on interactive mode for live updates
fig, ax = plt.subplots()
while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    # Resize uploaded mask to match frame size if necessary
    if uploaded_mask.shape != frame1.shape[:2]:
        uploaded_mask = cv2.resize(uploaded_mask, (frame1.shape[1], frame1.shape[0]))
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Feature matching
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(prev_descriptors, descriptors)
    matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) > 10:  # Ensure we have enough matches to find a homography
        src_pts = np.float32([prev_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if M is not None:
            h, w = gray.shape
            gray = cv2.warpPerspective(gray, M, (w, h))
            
            # Calculate and display the shift in x and y directions
            dx, dy = M[0, 2], M[1, 2]
            #print(f"Shift in x: {dx:.2f}, Shift in y: {dy:.2f}")
            
           
            shifts_x.append(dx)
            shifts_y.append(dy)

        

    # Update keypoints and descriptors for the next frame
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Apply MOG2 to get the foreground mask
    fgMask = backSub.apply(gray)

    # Initialize avg_frame if it has not been initialized
    if avg_frame is None:
        avg_frame = np.float32(gray)

    # Calculate the weighted average
    cv2.accumulateWeighted(gray, avg_frame, alpha)
    avg_gray = cv2.convertScaleAbs(avg_frame)

    # Thresholding the difference between the current frame and the running average
    _, avg_mask = cv2.threshold(cv2.absdiff(gray, avg_gray), 30, 255, cv2.THRESH_BINARY)

    # Combine MOG2 mask and weighted average mask to refine the foreground mask
    combined_mask = cv2.bitwise_and(fgMask, avg_mask)
    # Now combine the uploaded mask with the combined_mask from video processing
    final_mask = cv2.bitwise_and(combined_mask, uploaded_mask)

    # Calculate dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Set the magnitude of optical flow to zero wherever the combined mask is zero
    magnitude[final_mask == 0] = 0

    # Visualization of the optical flow magnitude (optional)
    norm_mag = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imshow('Optical Flow Magnitude (Foreground)', norm_mag.astype(np.uint8))

    prev_gray = gray.copy()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Initialize plotting
plt.ion()  # Turn on interactive mode for live updates
fig, ax = plt.subplots()

# Your existing video processing setup here
# ...

shifts_x = []
shifts_y = []

while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    # Your existing video processing logic here
    # ...

    if M is not None:
        # Extract shifts and append to lists for plotting
        dx, dy = M[0, 2], M[1, 2]
        shifts_x.append(dx)
        shifts_y.append(dy)

       
        ax.clear()
        ax.plot(shifts_x, label='Shift in X')
        ax.plot(shifts_y, label='Shift in Y')
        ax.legend()
        plt.draw()
        plt.pause(0.01)  # Pause to ensure the plot updates

    # Your existing video processing logic here
    # ...

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
plt.ioff()  # Turn off interactive mode
plt.show()
